To dos
1) Add additional functions (For categorical datasets - Histogram)
2) Improve logic and UX of main script
3) Add checks whether image asset already exists  
4) Handle NAs
5) Generate preprocessing scripts for other datasets

In [1]:
import ee
# ee.Authenticate()
ee.Initialize()

In [2]:
from datetime import date
from datetime import timedelta
import os
import datetime
import pandas as pd
import ee
import geemap
import os
import eeDatabase_coreMethods as eedb_cor
import eeDatabase_collectionMethods as eedb_col

Map = geemap.Map()

## Define Parameters and Load datasets

In [3]:
# ------------------------------------- Define parameters -----------------------------------------------

# Define time period to export
start_date = datetime.datetime(2022, 3, 1)
end_date = datetime.datetime(2022, 5, 1)

# Define whether to initialize new image collection or append to existing image collection
process = 'initialize' 


# -------------------------------- Define input Image Collection ----------------------------------------

# Define input Image Collection
in_ic_path = "GRIDMET/DROUGHT"
in_ic = ee.ImageCollection(in_ic_path)

# Define variable to generate database table for
var_name = 'long_term_drought_blend'
in_var_type = 'continuous'


# ------------------------------- Define input Feature Collection ---------------------------------------

in_fc_path = 'projects/dri-apps/assets/blm-admin/BLM_Natl_Grazing_Allotment_Polygons_Simplified_clean'
in_fc = ee.FeatureCollection(in_fc_path)
land_unit = 'blm-natl-grazing-allotment-polygons'

# # Subset by geometry
# geometry = ee.Geometry.Polygon([[[-108.4020, 38.7855], [-108.4020, 39.6080], [-109.1823, 39.6080], [-109.1823, 38.7855]]], None, False);
# in_fc = in_fc.filterBounds(geometry)
# # Specify ID property
# in_fc_id = "ALLOT_ID"

# Use full Feature Collection
in_fc = in_fc
# Specify ID property
in_fc_id = "ALLOT_ID"


# --------------------------------- Define database output path -----------------------------------------
var_name_exp = var_name.replace('_', '-')
out_path = f"projects/dri-apps/assets/blm-database/{var_name_exp}"


## Initialize the EE Image Collection and add ID band

In [4]:
# Apply ID image function to input feature collection
out_list = eedb_cor.generate_id_i(in_fc, in_fc_id, land_unit, in_fc_path, in_ic_path, in_var_type, var_name)
out_i = ee.Image(out_list.get(0))
out_fc = ee.FeatureCollection(out_list.get(1))

# Generate empty Image Collection asset to append images
os.system(f"earthengine create collection {out_path}")

# Export ID image to new Image Collection
task = ee.batch.Export.image.toAsset(
    image = out_i,
    description = f'Initialize - {var_name_exp} - id',
    assetId = out_path + '/id',
    region = out_fc.geometry().buffer(20),
    scale = 22.264,
    maxPixels = 1e13)
task.start()

Asset projects/dri-apps/assets/blm-database/long-term-drought-blend already exists.


## Pre-process image collection for generating database

In [4]:
# Run function to pre-process the GridMET drought data
in_i = eedb_col.preprocess_gm_drought(in_ic, var_name, start_date, end_date)

# Get list of date strings from image
in_dates = in_i.bandNames().getInfo()
# print(in_dates)

['20220301', '20220306', '20220311', '20220316', '20220321', '20220326', '20220331', '20220405', '20220410', '20220415', '20220420', '20220425', '20220430']


## Loop over dates to produce database image

In [30]:
# Iterate over in_dates with functions
for in_date in in_dates:
    print(in_date)
    
    # Select date band
    in_i_date = in_i.select([in_date])
    
    # Run function to get time-series statistics for input feature collection for continuous variables
    out_fc = img_to_pts_continuous(in_fc, out_fc, land_unit, in_fc_path, in_fc_id, in_ic_path, in_var_type, var_name)
    
    # Convert centroid time-series to image collection time-series
    out_i_date = pts_to_img_continuous(out_fc)
    
    # Export the image
    export_img(out_i_date, out_path, out_fc, var_name_exp, in_date)

20220105
20220110
20220115
20220120
20220125
20220130
20220204
20220209
20220214
20220219
20220224
